#Importante: en todos los algoritmos revisar las rutas de las carpeta y adecuar losa algoritmo para uso en diferentes escenarios.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **Archivos necesarios para el preprocesamiento**
import gdown
import os
def descarga(diccionario, directorio):
    # Extrae el nombre del archivo y el enlace del diccionario
    nombre_archivo = diccionario['nombre']
    enlace = diccionario['enlace']

    # Extrae el hash del enlace
    file_id = enlace.split('/')[-2]

    # Genera la URL de descarga directa
    url_descarga = f'https://drive.google.com/uc?id={file_id}'

    # Directorio de destino para la descarga
    directorio_destino = directorio

    # Descarga el archivo en el directorio de destino con el nombre original
    archivo_descargado = os.path.join(directorio_destino, nombre_archivo)
    gdown.download(url_descarga, output=archivo_descargado, quiet=False)

    print(f'Archivo descargado desde {enlace} como {nombre_archivo}')

# Lista de enlaces y nombres de archivo en forma de diccionarios
enlaces = [
    {'nombre': 'requirements.txt', 'enlace': 'https://drive.google.com/file/d/1sFLw-0OUD76Xnd1MPzr3Xau6LNGu3Qjw/view?usp=share_link'},
    {'nombre': 'mni_icbm152_t1_tal_nlin_sym_09a.nii', 'enlace': 'https://drive.google.com/file/d/19oV07-H2HTMFSvkcWisysLmW8aKbV2lh/view?usp=share_link'},
    {'nombre': 'helpers.py', 'enlace': 'https://drive.google.com/file/d/1puT1QhGpWDB1C8EXY7QwGPWuqdOAPrfG/view?usp=share_link'},
    {'nombre': 'aal.nii.gz', 'enlace': 'https://drive.google.com/file/d/15C5-FN8GgE9GSpahep54k9mk83GU6oVf/view?usp=sharing'},
    {'nombre': 'carpeta', 'enlace': 'https://drive.google.com/drive/folders/1rcjw19iFrO_rbM-5hy5Bep4F-yrRiwMu?usp=drive_link'}
]
#Cambiar el directorio a donde se quiere que se descargen los archivos:
directorio = '/content/sample_data'

for diccionario in enlaces:
    descarga(diccionario, directorio)

In [ ]:
#@title **Instalar librerias necesarias para el preprocesamiento**

!pip install nilearn
!pip install nibabel
!pip install -r /content/sample_data/requirements.txt

In [ ]:
#@title **Cargar librerias necesarias para el preprocesamiento**

import os
import sys
sys.path.append('/content/sample_data')
from helpers import *

from nilearn import datasets
from nilearn import image as nimg
from nilearn import plotting as nplot
import nibabel as nib
from nilearn.maskers import nifti_labels_masker
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import seaborn as sns
import numpy as np
import ants
import SimpleITK as sitk
from antspynet.utilities import brain_extraction
print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

In [ ]:
#@title **Algortimo par hacer normalizacion y skullstriping**

# Definir una funcion que haga todo el procesos de principio a fin, las entradas van a ser la ruta de la carpeta de los archivos a procesar y la ruta de la carpeta donde se van a guardar los archivos
import os
import ants
import SimpleITK as sitk


def preprocessing_anat(anat_path, anat_path_result, template_path):
    try:
        if not os.path.exists(anat_path_result):
            os.makedirs(anat_path_result)

        # Obtener la ruta del archivo de anat .nii.gz
        anat_file_path = os.listdir(anat_path)
        anat_file_path = [file for file in anat_file_path if file.endswith(".nii.gz")]
        anat_file = []

        #the reorient parameter set as IAL means: "Image Anterior Left"
        for file in anat_file_path:
            anat_file.append(ants.image_read(os.path.join(anat_path, file), reorient="IAL"))

        # # El sitk sirve para tener mayor precisión en los datos
        # anat_file_float = []

        # for file in anat_file_path:
        #     anat_file_float.append(sitk.ReadImage(os.path.join(anat_path, file), sitk.sitkFloat32))

        template_img_path = template_path
        template_img_ants = ants.image_read(template_img_path, reorient="IAL")

        #here the registration process is made
        transformation = ants.registration(
            fixed=template_img_ants,
            moving=anat_file[0],
            type_of_transform="SyN", #simetric normalization (useful for anatomical or functional scans)
            verbose=True,
        )

        #warpedmovout: move the image out to the fixed image
        registered_img_ants = transformation["warpedmovout"]
        prob_brain_mask = brain_extraction(registered_img_ants, modality="t1", verbose=True)
        brain_mask = ants.get_mask(prob_brain_mask, low_thresh=0.5)
        masked = ants.mask_image(registered_img_ants, brain_mask)
        #Segmentacion


        for i in range(len(anat_file)):
            processed_file = os.path.join(anat_path_result, anat_file_path[i].split(".")[0] + "_preprocessed.nii.gz",)
            mask_file = os.path.join(anat_path_result, anat_file_path[i].split(".")[0] + "_preprocessed_brain_mask.nii.gz",)
            mask_file_applied = os.path.join(anat_path_result, anat_file_path[i].split(".")[0] + "_preprocessed_brain_mask_applied.nii.gz",)

            if not os.path.exists(processed_file):
                # El archivo NO existe, entonces lo creamos.
                ants.image_write(registered_img_ants, processed_file)
                print(f"Archivo {anat_file_path[i]} creado en la carpeta {anat_path_result}")

            if not os.path.exists(mask_file):
                # La máscara NO existe, entonces la creamos.
                ants.image_write(brain_mask, mask_file)
                print(f"Máscara del cerebro para {anat_file_path[i]} creada en la carpeta {anat_path_result}")

            if not os.path.exists(mask_file_applied):
                # La máscara NO existe, entonces la creamos.
                ants.image_write(masked, mask_file_applied)
                print(f"Máscara del cerebro para {anat_file_path[i]} creada en la carpeta {anat_path_result}")

            if os.path.exists(processed_file) and os.path.exists(mask_file) and os.path.exists(mask_file_applied):
                # Tanto el archivo como la máscara YA existen, mostramos un mensaje de aviso.
                print(f"Archivo {anat_file_path[i]}, su máscara y su mascara aplicada ya existen en la carpeta {anat_path_result}")

    except Exception as e:
        # Capturamos cualquier excepción y mostramos un mensaje de error.
        print(f"Error: {e}")


In [ ]:
#@title **Uso: ingresar el path de la carpeta donde estan los archivos anat, y agregar la ruta para guardar el resultado, agregar la ruta de un template para hacer la normalizacion**

anat_path = "/content/drive/MyDrive/neurocon/data/raw/parkinson/sub-patient032030/anat"
anat_path_result = "/content/sample_data"
template_path = "/content/sample_data/aal.nii.gz"
preprocessing_anat(anat_path, anat_path_result, template_path)

In [ ]:
#@title **Visualizacion de archivos**

#Uso: Se proporciona el la ruta del archivo normalizado y la ruta del archivo con la mascara aplicada

# Ver un archivo y su mascara

#Obtener la ruta del archivo de anat .nii.gz
anat_file_origin = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/raw/parkinson/sub-patient032030/anat/sub-patient032030_T1w.nii.gz",
    reorient="IAL")
anat_file = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/preprocessed/parkinson/sub-patient032030/anat/sub-patient032030_T1w_preprocessed.nii.gz",
    reorient="IAL")
mask_file_aplied = ants.image_read(
    "/content/drive/MyDrive/neurocon/data/preprocessed/parkinson/sub-patient032030/anat/sub-patient032030_T1w_preprocessed_brain_mask_applied.nii.gz",
    reorient="IAL")
# explore_3D_array_with_mask_contour(anat_file.numpy(), mask_file.numpy())
#print(anat_file)
print(anat_file_origin)
print(anat_file)
print(mask_file_aplied)

explore_3D_array_comparison(
    arr_before=anat_file.numpy(),
    arr_after=mask_file_aplied.numpy(),
    cmap="gray",
)
explore_3D_array(anat_file_origin.numpy())

In [ ]:
#@title **Registro hace un atlas para un solo archivo**

img_path = '/content/drive/MyDrive/neurocon/data/preprocessed/control/sub-control032014/anat/sub-control032014_T1w_preprocessed.nii.gz'
img = nib.load(atlas_path)
atlas_path = '/content/PD25-subcortical-1mm.nii.gz'
atlas = nib.load(atlas_path)

print('Size of functional image:', img.shape)
print('Size of atlas image:', atlas.shape)

resultado = nimg.resample_to_img(atlas, img, interpolation = 'nearest')
#resultado.to_filename(ruta_directorio + nombre_archivo)

nplot.plot_roi(resultado, img.slicer[:, :, 54:55])

In [ ]:
#@title **Registro hace un atlas para varios archivos**
import os
import nibabel as nib
import nilearn.image as nimg

# Ruta de la carpeta que contiene los archivos preprocesados
base_folder = '/content/drive/MyDrive/neurocon/data/preprocessed'

# Ruta de la imagen del atlas (carga fuera del bucle)
atlas_path = '/content/PD25-subcortical-1mm.nii.gz'
atlas = nib.load(atlas_path)

# Itera a través de todas las carpetas dentro de la carpeta 'preprocessed'
for root, dirs, files in os.walk(base_folder):
    for file in files:
        # Verifica si el archivo sigue la nomenclatura deseada
        if file.endswith('T1w_preprocessed.nii.gz'):
            # Ruta completa del archivo actual
            img_path = os.path.join(root, file)

            # Carga la imagen actual
            img = nib.load(img_path)

            # Imprime tamaños de las imágenes para verificar
            print('Size of functional image:', img.shape)
            print('Size of atlas image:', atlas.shape)

            # Realiza la resample_to_img
            resultado = nimg.resample_to_img(atlas, img, interpolation='nearest')

            # Modifica el nombre del archivo para incluir '_parcelation'
            new_file_name = file.replace('T1w_preprocessed.nii.gz', 'T1w_preprocessed_parcelation.nii.gz')

            # Ruta completa para guardar el resultado en la misma carpeta
            output_path = os.path.join(root, new_file_name)

            # Guarda el resultado en la misma carpeta que el archivo original
            nib.save(resultado, output_path)

            # Imprime la ruta donde se guardó el resultado
            print(f'Resultado guardado en: {output_path}')